In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

!mkdir -p {DATA_DIR} {NAVEC_DIR} {MODEL_DIR}
s3 = S3()

In [ ]:
if not exists(NERUS):
    s3.download(S3_NERUS, NERUS)
    
if not exists(NAVEC):
    !wget {NAVEC_URL} -O {NAVEC}

In [ ]:
navec = Navec.load(NAVEC)

In [ ]:
words_vocab = Vocab(navec.vocab.words)
shapes_vocab = Vocab([PAD] + SHAPES)
tags_vocab = BIOTagsVocab([PER, LOC, ORG])

In [ ]:
torch.manual_seed(SEED)
seed(SEED)

In [ ]:
word = NavecEmbedding(navec)
shape = Embedding(
    vocab_size=len(shapes_vocab),
    dim=SHAPE_DIM,
    pad_id=shapes_vocab.pad_id
)
emb = TagEmbedding(word, shape)
encoder = TagEncoder(
    input_dim=emb.dim,
    layer_dims=LAYER_DIMS,
    kernel_size=KERNEL_SIZE,
)
ner = NERHead(encoder.dim, len(tags_vocab))
model = NER(emb, encoder, ner)

model = model.to(DEVICE)

In [ ]:
lines = load_gz_lines(NERUS)
lines = log_progress(lines, total=NERUS_TOTAL)
items = parse_jl(lines)

markups = (SpanMarkup.from_json(_) for _ in items)
markups = (_.to_bio(list(tokenize(_.text))) for _ in markups)

encode = TagTrainEncoder(
    words_vocab, shapes_vocab, tags_vocab,
    seq_len=256,
    batch_size=64,
    shuffle_size=1000,
)
batches = encode(markups)
# full lenta is ~10k batches
# epoch takes ~1h, on 2nd epoch loss expodes
# ~5k seems to saturate acc
batches = head(batches, 5000)
batches = [_.to(DEVICE) for _ in batches]

size = 10  # ~750 articles
batches = {
    TEST: batches[:size],
    TRAIN: batches[size:]
}

In [ ]:
board = MultiBoard([
    TensorBoard(BOARD_NAME, RUNS_DIR),
    LogBoard()
])
boards = {
    TRAIN: board.section(TRAIN_BOARD),
    TEST: board.section(TEST_BOARD),
}

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, LR_GAMMA)

In [ ]:
meters = {
    TRAIN: NERScoreMeter(),
    TEST: NERScoreMeter(),
}

for epoch in log_progress(range(EPOCHS)):
    model.train()
    for batch in log_progress(batches[TRAIN], leave=False):
        optimizer.zero_grad()
        batch = process_batch(model, ner.crf, batch)
        batch.loss.backward()
        optimizer.step()
    
        score = NERBatchScore(batch.loss)
        meters[TRAIN].add(score)

    meters[TRAIN].write(boards[TRAIN])
    meters[TRAIN].reset()

    model.eval()
    with torch.no_grad():
        for batch in log_progress(batches[TEST], leave=False, desc=TEST):
            batch = process_batch(model, ner.crf, batch)
            batch.pred = ner.crf.decode(batch.pred)
            score = score_ner_batch(batch, tags_vocab)
            meters[TEST].add(score)

        meters[TEST].write(boards[TEST])
        meters[TEST].reset()
    
    scheduler.step()
    board.step()

In [ ]:
# [2020-04-11 09:35:47]    0 7.3741 01_train/01_loss
# [2020-04-11 09:36:09]    0 4.2063 02_test/01_loss
# [2020-04-11 09:36:09]    0 0.9776 02_test/02_PER
# [2020-04-11 09:36:09]    0 0.9691 02_test/03_LOC
# [2020-04-11 09:36:09]    0 0.9486 02_test/04_ORG
# [2020-04-11 10:09:54]    1 4.5356 01_train/01_loss
# [2020-04-11 10:10:18]    1 3.8757 02_test/01_loss
# [2020-04-11 10:10:18]    1 0.9791 02_test/02_PER
# [2020-04-11 10:10:18]    1 0.9711 02_test/03_LOC
# [2020-04-11 10:10:18]    1 0.9494 02_test/04_ORG
# [2020-04-11 10:44:08]    2 4.1505 01_train/01_loss

# [2020-04-11 10:44:32]    2 3.7297 02_test/01_loss
# [2020-04-11 10:44:32]    2 0.9805 02_test/02_PER
# [2020-04-11 10:44:32]    2 0.9729 02_test/03_LOC
# [2020-04-11 10:44:32]    2 0.9529 02_test/04_ORG

In [ ]:
# model.emb.shape.dump(MODEL_SHAPE)
# model.encoder.dump(MODEL_ENCODER)
# model.ner.dump(MODEL_NER)

# s3.upload(MODEL_SHAPE, S3_MODEL_SHAPE)
# s3.upload(MODEL_ENCODER, S3_MODEL_ENCODER)
# s3.upload(MODEL_NER, S3_MODEL_NER)